### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,"{'rank_eps001': 464, 'best_metric_val': 0.1575...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",1mf6593n
1,"{'reg_loss': 15.066952596664429, 'rank_eps001'...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",2s8ksd8i
2,"{'_step': 23000, 'neg_sim': 0.0940513908863067...","{'env': {'game': 'Hero', 'seed': 42, 'type': '...",1k1krv88
3,"{'obs_loss': 0.12343119064718484, 'pos_neg_dif...","{'env': {'game': 'Frostbite', 'seed': 42, 'typ...",1y9n44ao
4,"{'act_f1': 0.1084116025179768, '_timestamp': 1...","{'env': {'game': 'Jamesbond', 'seed': 42, 'typ...",1ygglb86
...,...,...,...
1853,"{'negative_sim': 0.04744461342226714, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1854,"{'pos_neg_diff': 0.4670497511327267, 'positive...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1855,"{'pos_neg_diff': 0.8050060951709748, 'positive...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1856,"{'_runtime': 17708, '_timestamp': 1659279862, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'gpt_cont_demon_nproj'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_243354/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
53,"{'act_loss': 0.30039975168259164, 'obs_loss': ...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",2rx2mmyp,UpNDown
54,"{'_step': 234370, '_runtime': 77957.6283950805...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",3sgb0e2x,Seaquest
73,"{'act_acc': 0.8473364061456385, 'reward_f1': 0...","{'env': {'game': 'PrivateEye', 'seed': 42, 'ty...",17eodtwu,PrivateEye
74,"{'reward_f1': 0.8843069873997709, '_timestamp'...","{'env': {'game': 'Qbert', 'seed': 42, 'type': ...",pxdeq9x1,Qbert
75,"{'obs_loss': 0.3426476202521544, 'rank_eps1': ...","{'env': {'game': 'RoadRunner', 'seed': 42, 'ty...",vd5uovvn,RoadRunner
76,"{'_wandb': {'runtime': 77356}, 'act_f1': 0.335...","{'env': {'game': 'Pong', 'seed': 42, 'type': '...",3na1r57h,Pong
98,"{'obs_loss': 0.18452037018812123, '_timestamp'...","{'env': {'game': 'Krull', 'seed': 42, 'type': ...",24zi5j71,Krull
99,"{'min_grad_norm': 7.523253538361434e-09, 'loss...","{'env': {'game': 'MsPacman', 'seed': 42, 'type...",25phpg4m,MsPacman
100,"{'_wandb': {'runtime': 77187}, 'pos_sim': 0.95...","{'env': {'game': 'KungFuMaster', 'seed': 42, '...",341aqoog,KungFuMaster
101,"{'act_loss': 0.4708218089125667, 'obs_loss': 0...","{'env': {'game': 'Kangaroo', 'seed': 42, 'type...",3m4efmjh,Kangaroo


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown 2rx2mmyp
Seaquest 3sgb0e2x
PrivateEye 17eodtwu
Qbert pxdeq9x1
RoadRunner vd5uovvn
Pong 3na1r57h
Krull 24zi5j71
MsPacman 25phpg4m
KungFuMaster 341aqoog
Kangaroo 3m4efmjh
Gopher 2b4ta3px
Frostbite rbw2skdw
Jamesbond 2fp80r43
Hero 3lsg9i74
CrazyClimber wgtsn6lc
ChopperCommand 1fenh2wz
Freeway 3q6wf1uy
DemonAttack x2qoxo6u
BankHeist 1tglcp19
Boxing 1yjp41zy
Breakout cbvsbyk5
BattleZone w9lckysr
Alien 1ucvzs0e
Asterix 1uo49buz
Assault 2bfctcrw
Amidar xfv4hmbq


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)